In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Flatten, Dense, Dropout, Reshape
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

2024-05-12 17:46:53.235716: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 17:46:54.582560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_dir = '/teamspace/studios/this_studio/Augmented Data'
image_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.jpeg')]

In [3]:
labels = [0 if 'Benign' in file else 1 for file in image_files]

In [4]:
train_files, test_val_files, train_labels, test_val_labels = train_test_split(image_files, labels, test_size=0.4, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(test_val_files, test_val_labels, test_size=0.5, random_state=42)

In [5]:
train_labels[:5]

[1, 1, 1, 1, 1]

In [6]:
# Convert labels to string type
# train_labels_str = [str(label) for label in train_labels]
# val_labels_str = [str(label) for label in val_labels]
# test_labels_str = [str(label) for label in test_labels]
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

train_labels_str =  train_labels
val_labels_str = val_labels
test_labels_str = test_labels

train_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_files, 'class': train_labels_str}),
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='raw'

)

val_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': val_files, 'class': val_labels_str}),
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='raw'
)

test_generator = datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_files, 'class': test_labels_str}),
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='raw'

)


Found 4372 validated image filenames.
Found 1457 validated image filenames.
Found 1458 validated image filenames.


In [7]:
# model = Sequential([
#     LSTM(64, input_shape=(None, 150*150*3)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')
# ])

model = Sequential([
    Reshape((150, 450), input_shape=(150, 150, 3)),  # Reshape from (150, 150, 3) to (150, 450)
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model = Sequential([
    Reshape((150, 450), input_shape=(150, 150, 3)),
    LSTM(64),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # L2 regularization
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

early_stop = EarlyStopping(monitor='val_loss', patience=5)  # Early stopping

In [9]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Fit the model
history = model.fit(train_generator, epochs=50, validation_data=val_generator)

Epoch 1/50


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


137/137 ━━━━━━━━━━━━━━━━━━━━ 15s 92ms/step - accuracy: 0.6492 - loss: 0.7087 - val_accuracy: 0.6438 - val_loss: 0.6666
Epoch 2/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.6568 - loss: 0.6600 - val_accuracy: 0.6438 - val_loss: 0.6393
Epoch 3/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.6656 - loss: 0.6302 - val_accuracy: 0.6438 - val_loss: 0.6176
Epoch 4/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - accuracy: 0.6456 - loss: 0.6235 - val_accuracy: 0.6362 - val_loss: 0.6040
Epoch 5/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.6670 - loss: 0.5965 - val_accuracy: 0.6644 - val_loss: 0.5980
Epoch 6/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.6590 - loss: 0.5934 - val_accuracy: 0.6438 - val_loss: 0.5888
Epoch 7/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - accuracy: 0.6680 - loss: 0.5784 - val_accuracy: 0.6644 - val_loss: 0.5812
Epoch 8/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - accuracy: 0.6689 - loss: 0.5603 - val_accurac

In [11]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.6533 - loss: 0.6753
Test Loss: 0.6946873068809509, Test Accuracy: 0.659122109413147


In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Predict probabilities for the test set
y_pred_proba = model.predict(test_generator)

# Convert probabilities to binary predictions
y_pred = (y_pred_proba > 0.5).astype(int)

# Convert y_pred to 1D array
y_pred = y_pred.squeeze()

# Compute metrics
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
accuracy = accuracy_score(test_labels, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step
Precision: 0.6412315930388219
Recall: 0.5117521367521367
F1 Score: 0.5692216280451574
Accuracy: 0.5027434842249657


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('/teamspace/studios/this_studio/Saved Models/model.h5')

# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions
def predict_image(image_path):
    preprocessed_img = preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)
    if prediction < 0.5:
        return 'Benign'
    else:
        return 'Malignant'

# Upload a sample image and specify its path
uploaded_image_path = '/teamspace/studios/this_studio/Augmented Data/aug_Benign_0_8182.jpeg'  # Modify this with the path to your uploaded image

# Make a prediction
prediction = predict_image(uploaded_image_path)
print('Prediction:', prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Prediction: Benign


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('/teamspace/studios/this_studio/Saved Models/model.h5')

# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions
def predict_image(image_path):
    preprocessed_img = preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)
    if prediction < 0.5:
        return 'Benign'
    else:
        return 'Malignant'

# Upload a sample image and specify its path
uploaded_image_path = '/teamspace/studios/this_studio/Augmented Data/aug_Malignant_0_862.jpeg'  # Modify this with the path to your uploaded image

# Make a prediction
prediction = predict_image(uploaded_image_path)
print('Prediction:', prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Prediction: Malignant
